# Type-hint different number of return values with python mypy
* https://stackoverflow.com/questions/73109331/type-hint-different-number-of-return-values-with-python-mypy

In [1]:
%load_ext nb_mypy

Version 1.0.3


In [42]:
from typing import Any, Union, Dict, Tuple, cast, overload, Literal, get_args, TypeGuard

In [3]:
def f(x : bool) -> Union[Tuple[int, int], Tuple[int, int, int]]:
    if x:
        return 1, 2, 3
    return 1, 2

In [16]:
f_return = f(x=False)
assert len(f_return) == 2
a, b = f_return
a, b

(1, 2)

In [17]:
f_return = f(x=True)
assert len(f_return) == 3
x, y, z = f_return
x, y, z

(1, 2, 3)

In [20]:
a, b = cast(Tuple[int, int], f(x=False))
a, b

(1, 2)

In [21]:
x, y, z = cast(Tuple[int, int, int], f(x=True))
x, y, z

(1, 2, 3)

In [9]:
def f1(x: bool) -> Union[int, str]:
    if x:
        return 1
    return 'string'

In [10]:
a: int
a = f1(True)

<cell>2: error: Incompatible types in assignment (expression has type "Union[int, str]", variable has type "int")


In [22]:
len(Tuple[int, bool])

<cell>1: error: Argument 1 to "len" has incompatible type "object"; expected "Sized"


TypeError: object of type '_GenericAlias' has no len()

In [45]:
def f(x: str) -> tuple[int, bool]|tuple[int, str, float]:
    if 'something' in x:
        return 1, 'this', 3.0
    return 1, True

In [46]:
def is_tuple_int_bool(tuple_arg: tuple) -> TypeGuard[tuple[int, bool]]:
    return len(tuple_arg) == 2 and isinstance(tuple_arg[0], int) and isinstance(tuple_arg[1], bool)

def is_tuple_int_str_float(tuple_arg: tuple) -> TypeGuard[tuple[int, str, float]]:
    return len(tuple_arg) == 3 and isinstance(tuple_arg[0], int) and isinstance(tuple_arg[1], str) and isinstance(tuple_arg[2], float)

In [48]:
return_value = f('something')
return_value = f('else')
if is_tuple_int_bool(return_value):
    a, b = return_value
    print(a, b)
elif is_tuple_int_str_float(return_value):
    x, y, z = return_value
    print(x, y, z)
else:
    raise ValueError('Unexpected type')

1 True


In [49]:
return_value = f('something')
assert is_tuple_int_str_float(return_value)
x, y, z = return_value
x, y, z

(1, 'this', 3.0)

In [38]:
CASE2 = 'this is something'

@overload
def f(x: Literal[CASE2]) -> Tuple[int, str, float]:
    ...

@overload
def f(x: Literal['this is not']) -> Tuple[int, bool]:
    ...
    
def f(x: str) -> Tuple:
    if x == 'this is something':
        return 1, 'this', 3.0
    elif x == 'this is not':
        return 1, True
    else:
        raise ValueError

<cell>4: error: Parameter 1 of Literal[...] is invalid
<cell>4: error: Variable "__main__.CASE2" is not valid as a type
<cell>4: note: See https://mypy.readthedocs.io/en/stable/common_issues.html#variables-vs-type-aliases
<cell>8: error: Overloaded function signature 2 will never be matched: signature 1's parameter type(s) are the same or broader


In [36]:
a, b = f('this is not')
a, b

(1, True)

In [37]:
x, y, z = f('this is something')
x, y, z

(1, 'this', 3.0)

In [41]:
get_args(Literal['add'])

('add',)